In [5]:
import pandas as pd
import time
import akshare as ak

In [10]:
# this func will get performance report of all listed companys
def getdata() :
    # 创建一个基数列表，注意列表是不包含最后一个年份的。
    l1 = list(range(20100000, 20230000, 10000))
    l2 = [331, 630, 930, 1231]
    qs = [ m+n for m in l1 for n in l2]
    yjbg = pd.DataFrame()
    for q in qs:
        df = ak.stock_yjbb_em(date=str(q))
        df['qs'] = str(q) # add a new col of period
        yjbg = pd.concat([yjbg,df], ignore_index=True)
        time.sleep(2)
        print(q)
    yjbg.to_csv('../../yjbg2022q4.csv', index=False)

    return yjbg

In [15]:
def showProfile(df) :
    col_profile = pd.concat([df.dtypes, df.isna().sum()/len(df)*100, df.sample().T], axis=1 ).reset_index()
    col_profile.columns = ['列名', '类型', '缺失比%','样例']
    
    return (col_profile)

# 数据集的准备

## 下载数据

- 从akshare上下载业绩报告数据。
- 从tushare下载上市公司基本情况数据。
- 两个数据集的基本情况如下。

In [33]:
df = pd.read_pickle('../../yjbg2022q4.pkl.gz')
showProfile(df)

,列名,类型,缺失比%,样例
0,股票代码,object,0.000000,600819
1,股票简称,object,0.000000,耀皮玻璃
2,每股收益,float64,1.772489,0.1
3,营业收入-营业收入,float64,0.188974,2807338887.07
4,净利润-净利润,float64,0.026506,89921295.2
5,每股净资产,float64,4.812289,3.292242
6,净资产收益率,float64,2.447307,2.95
7,每股经营现金流量,float64,4.056393,0.076614
8,销售毛利率,float64,2.401467,17.700716
9,最新公告日期,object,0.000000,2019-10-30


In [34]:
tu = pd.read_pickle('../../tubasic.pkl')
showProfile(tu)

,列名,类型,缺失比%,样例
0,ts_code,object,0.000000,002910.SZ
1,symbol,object,0.000000,002910
2,name,object,0.000000,庄园牧场
3,area,object,3.664211,甘肃
4,industry,object,3.664211,乳制品
5,fullname,object,0.000000,兰州庄园牧场股份有限公司
6,enname,object,0.000000,"Lanzhou Zhuangyuan Pasture Co., Ltd."
7,cnspell,object,0.000000,zymc
8,market,object,0.000000,中小板
9,exchange,object,0.000000,SZSE


## 合并数据

- 把基本信息中的['area', 'industry', 'market', 'exchange', 'list_status','list_date', 'delist_date', 'is_hs']合并到业绩报告中去。
- 业绩报告包括了新三板的，而股票信息则只含有北上深三个交易所的信息。
- 保留业绩报告的数据的情况下合并：left merge

In [36]:
tuu = tu[['symbol','area', 'industry', 'market', 'exchange', 'list_status','list_date', 'delist_date', 'is_hs']]
dfm = df.merge(tuu, how='left', left_on='股票代码', right_on='symbol')

## 整理一下数据

- 把市场和交易所的信息填好，北上深市以外的填写为其它。
- 把报告期分成年和季度。

In [47]:
# fill the exhange and market with other, and split the year and quarters
dfm[['market', 'exchange']] = dfm[['market', 'exchange']].fillna('其它')
dfm['Year'] = df['qs'].str[0:4]
dfm['Quarter'] = df['qs'].str[4:].replace({'0331':'Q1', '0630':'Q2', '0930':'Q3','1231':'Q4'})

In [55]:
showProfile(dfm)

,列名,类型,缺失比%,样例
0,股票代码,object,0.000000,830927
1,股票简称,object,0.000000,ST兆久成
2,每股收益,float64,1.772489,-0.43
3,营业收入-营业收入,float64,0.188974,77894649.3
4,净利润-净利润,float64,0.026506,-12783724.83
5,每股净资产,float64,4.812289,-0.39
6,净资产收益率,float64,2.447307,NaN
7,每股经营现金流量,float64,4.056393,-0.074653
8,销售毛利率,float64,2.401467,3.209326
9,最新公告日期,object,0.000000,2023-04-17


In [58]:
dfm.to_pickle('../../tidy2022.pkl.gz')